# YOLOv11 Implementation for Raspberry Pi

This notebook implements a lightweight YOLOv11 model optimized for deployment on Raspberry Pi 5, using NCNN for efficient inference.

In [ ]:
# Install required dependencies
!pip install torch torchvision numpy opencv-python ncnn

# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
import ncnn
import time
from tqdm.notebook import tqdm
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Model Architecture

Define the YOLOv11 model with a lightweight MobileNetV3-like backbone for efficient computation on Raspberry Pi.

In [ ]:
class YOLOLoss(nn.Module):
    def __init__(self):
        super(YOLOLoss, self).__init__()
        self.mse = nn.MSELoss(reduction='sum')
        
    def forward(self, predictions, targets):
        # Split predictions into box coordinates, objectness, and class predictions
        pred_boxes = predictions[..., :4]
        pred_obj = predictions[..., 4]
        pred_cls = predictions[..., 5:]
        
        # Split targets similarly
        target_boxes = targets[..., :4]
        target_obj = targets[..., 4]
        target_cls = targets[..., 5:]
        
        # Calculate losses
        box_loss = self.mse(pred_boxes, target_boxes)
        obj_loss = self.mse(pred_obj, target_obj)
        cls_loss = self.mse(pred_cls, target_cls)
        
        # Combine losses
        total_loss = box_loss + obj_loss + cls_loss
        return total_loss

## Model Conversion and Quantization

Implement functions for converting the model to NCNN format and quantizing it for efficient inference.

In [ ]:
def convert_to_ncnn(model, input_shape=(1, 3, 416, 416)):
    """Convert PyTorch model to NCNN format"""
    # Export to ONNX first
    dummy_input = torch.randn(input_shape)
    torch.onnx.export(model, dummy_input, "yolov11.onnx",
                     input_names=['input'],
                     output_names=['output'],
                     dynamic_axes={'input': {0: 'batch_size'},
                                 'output': {0: 'batch_size'}})
    
    # Convert ONNX to NCNN
    # Note: This requires the NCNN converter tool to be installed
    !ncnn2table yolov11.onnx yolov11.table
    !ncnn2param yolov11.onnx yolov11.param
    !ncnn2bin yolov11.onnx yolov11.bin

def quantize_model(model, calibration_data):
    """Quantize model for efficient inference"""
    model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
    torch.quantization.prepare(model, inplace=True)
    
    # Calibrate with sample data
    with torch.no_grad():
        for data in calibration_data:
            model(data)
    
    torch.quantization.convert(model, inplace=True)
    return model

## Model Training

Train the model on your dataset (COCO or custom dataset).


In [ ]:
# Install COCO API if not already installed
!pip install pycocotools

# Import COCO dataset utilities
from torchvision.datasets import CocoDetection
from torchvision import transforms
import torch.utils.data as data
import os

# Download COCO dataset
def download_coco_dataset():
    try:
        print("Downloading COCO dataset...")
        
        # Create directory for COCO dataset
        os.makedirs('./coco', exist_ok=True)
        
        # Check if files already exist
        if os.path.exists('./coco/val2017') and os.path.exists('./coco/annotations'):
            print("COCO dataset already exists!")
            return
        
        # Download COCO 2017 validation set
        !wget http://images.cocodataset.org/zips/val2017.zip
        !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
        
        print("Extracting files...")
        # Unzip the files
        !unzip -q val2017.zip -d ./coco/
        !unzip -q annotations_trainval2017.zip -d ./coco/
        
        print("Cleaning up zip files...")
        # Clean up zip files
        !rm val2017.zip
        !rm annotations_trainval2017.zip
        
        print("COCO dataset download complete!")
    except Exception as e:
        print(f"Error downloading COCO dataset: {e}")
        raise

download_coco_dataset()

# Setup COCO dataset
def setup_coco_dataset():
    # Create directory for COCO dataset
    os.makedirs('./coco', exist_ok=True)
    
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Load COCO validation set (smaller than training set for faster processing)
    dataset = CocoDetection(
        root='./coco/val2017',
        annFile='./coco/annotations/instances_val2017.json',
        transform=transform
    )
    
    return dataset

# Create data loader
def create_data_loader(dataset, batch_size=8):
    return data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4
    )

# Prepare calibration data
def prepare_calibration_data(dataset, num_images=100):
    calibration_data = []
    for i in range(min(num_images, len(dataset))):
        img, _ = dataset[i]
        calibration_data.append(img.unsqueeze(0).to(device))
    return calibration_data

# Initialize model and dataset
model = YOLOv11(num_classes=80).to(device)  # COCO has 80 classes
dataset = setup_coco_dataset()
data_loader = create_data_loader(dataset)

# Prepare calibration data
calibration_data = prepare_calibration_data(dataset)

# Training loop
def train_model(model, data_loader, num_epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    criterion = YOLOLoss()
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        # Add tqdm here for the data_loader loop
        for batch_idx, (images, targets) in enumerate(tqdm(data_loader, desc=f'Epoch {epoch+1}/{num_epochs}')):
            images = images.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            if batch_idx % 10 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(data_loader)}, Loss: {loss.item():.4f}')
            
        avg_loss = total_loss / len(data_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}')
        
        # Update learning rate based on average loss
        scheduler.step(avg_loss)
        
        # Save checkpoint after each epoch
        save_model(model, optimizer, epoch, avg_loss, f'yolov11_epoch_{epoch+1}.pth')

# Train the model
train_model(model, data_loader)

# Model evaluation
def evaluate_model(model, data_loader):
    model.eval()
    total_loss = 0
    criterion = YOLOLoss()
    
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(data_loader):
            images = images.to(device)
            targets = targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            if batch_idx % 10 == 0:
                print(f'Evaluation Batch {batch_idx}/{len(data_loader)}, Loss: {loss.item():.4f}')
    
    avg_loss = total_loss / len(data_loader)
    print(f'Evaluation Average Loss: {avg_loss:.4f}')
    return avg_loss

# Evaluate the model
evaluation_loss = evaluate_model(model, data_loader)

# Save the trained model
def save_model(model, optimizer, epoch, loss, path='yolov11_trained.pth'):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, path)
    print(f'Model saved to {path}')

# Save the model after training
save_model(model, optimizer, num_epochs, evaluation_loss)

# Quantize model
quantized_model = quantize_model(model, calibration_data)

# Convert to NCNN
convert_to_ncnn(quantized_model)

## Model Deployment

Create a detector class for running inference on the Raspberry Pi.

In [ ]:
class YOLOv11Detector:
    def __init__(self, model_path, conf_threshold=0.25, nms_threshold=0.45):
        self.conf_threshold = conf_threshold
        self.nms_threshold = nms_threshold
        
        # Load NCNN model
        self.net = ncnn.Net()
        self.net.load_param("yolov11.param")
        self.net.load_model("yolov11.bin")
        
        # Initialize NCNN extractor
        self.extractor = self.net.create_extractor()
        self.extractor.set_num_threads(4)  # Adjust based on Raspberry Pi capabilities
        
    def preprocess(self, image):
        """Preprocess image for inference"""
        # Resize to model input size
        img = cv2.resize(image, (416, 416))
        # Convert to RGB and normalize
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        # NCHW format
        img = img.transpose(2, 0, 1)
        return img
    
    def detect(self, image):
        """Run inference and return detections"""
        # Preprocess image
        blob = self.preprocess(image)
        
        # Run inference
        self.extractor.input("input", blob)
        output = self.extractor.extract("output")
        
        # Process detections
        boxes = []
        scores = []
        class_ids = []
        
        # Convert output to detections
        detections = output.reshape(-1, 85)  # 80 classes + 5 box coordinates
        
        # Filter by confidence
        mask = detections[:, 4] > self.conf_threshold
        detections = detections[mask]
        
        if len(detections) > 0:
            # Convert to boxes
            boxes = detections[:, :4]
            scores = detections[:, 4]
            class_ids = np.argmax(detections[:, 5:], axis=1)
            
            # Apply NMS
            indices = cv2.dnn.NMSBoxes(boxes, scores, self.conf_threshold, self.nms_threshold)
            
            if len(indices) > 0:
                boxes = boxes[indices]
                scores = scores[indices]
                class_ids = class_ids[indices]
        
        return boxes, scores, class_ids

## Testing on Raspberry Pi

After converting the model to NCNN format, you can copy the following files to your Raspberry Pi:
- yolov11.param
- yolov11.bin
Then use the YOLOv11Detector class to run inference on the Pi.